<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/experimental/bybit/bybit_basilbot_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# バジルさんの記事「Binanceで数か月運用していた高頻度botの紹介」を実装してみる。
https://note.com/kkngo/n/n13fb59bacc95?sub_rt=share_sb

## バックテスト　＆　フォーワードテスト

[テストネットによる本番運用はこちら](.//basilbot_real_run_testnet.ipynb)

## ロジック
バジルさんのロジックを引用すると以下のとおり

* 取引頻度
    約定履歴から作成した3秒足ごと
* エントリー/イグジット条件
    * 常に買いから入り、終値*0.9995など決め打ちでの買い指値（数字は仮）
    * イグジットも終値*1.0005など決め打ちでの売り指値（数字は仮）

* ピラミッディングやドテンの類はなし
* 手持ちのTUSD（2500TUSDくらい）分のBTCを買って売るだけ
* 4/27に大きく焼かれてからは、直近数時間の値動きが大きすぎるときはエントリーしないように対応
* 約定回数
    200～300回/日


これを今回の実装用に調整・具体化すると以下。

* 3秒毎にエントリーするかどうかを判定。

* エントリー条件
  * 前回のオーダーが残っていない（オーダー中および買いポジション(相当)がない）
  * 直前の値動きが大きくない
    * ３０分足を４本とり、平均の値幅が閾値以下。
* オーダーキャンセル条件
  * EXPIRE_TIME(600秒: 10分)以上約定しないオーダーはキャンセルする。
* 執行戦略
  * 買い
    * エントリー条件が揃ったら執行
    * 指値: 3秒足の終値 * (1-OFFSET)     // OFFSETは0.0005などの値
    * 数量: 0.001 BTC
  * 売り
    * 買いオーダーの約定が残っていたら執行
    * 指値：買いオーダー発行時の3秒足の終値 * (1+OFFSET)     // OFFSETは0.0005などの値
    * 数量：約定した買い注文と同数（=仮想敵なPosition）
    * その他：売りオーダーがExpireしてCancelされた場合、即、market_orderで投げうる。


In [1]:
# rbot拡張ライブラリのインストール（２回目は不要です）
# ! pip install --upgrade pip
! pip install --upgrade --index-url https://test.pypi.org/simple/ rbot  # test pypi からのインストール

Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 6.9 MB/s eta 0:00:00


In [2]:
# 関連ライブラリのインストール
# Polarsは0.20.0が必要です。
! pip install pyarrow
! pip install --upgrade polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.17.3
    Uninstalling polars-0.17.3:
      Successfully uninstalled polars-0.17.3
  Preparing metadata (setup.py) ... done
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7593 sha256=94815ba0ad3746d2b2850453b18cd403c231e217915fb5004b5488498b51e0c2
  Stored in directory: /root/.cache/pip/wheels/e0/d8/b3/6f83a04ab0ec00e691de794d108286bb0f8bcdf4ade19afb57
Successfully built json2html


# Agent（BOT）の実装

In [3]:
class BasilAgent:
    def __init__(self):
        self.OFFSET = 0.00_05
        self.EXPIRE_TIME = 600    # 600[sec] = 10[min]
        self.ORDER_SIZE = 0.01
        self.RANGE = 300

    def on_init(self, session):
        session.clock_interval_sec = 3     # 3秒ごとに on_clock を呼び出す

    def on_clock(self, session, clock):
        if session.expire_order(self.EXPIRE_TIME):         # 期限切れの注文をキャンセルする.
            return                                          # 期限切れがあればリターン

        if session.buy_orders or session.sell_orders:   # 既に注文がある場合はリターン
            return

        # 1時間足のレンジを計算してログに出力する。レンジが大きい場合はトレードしない。
        ohlcv1h = session.ohlcv(60*60, 4)
        range = (ohlcv1h['high']-ohlcv1h['low']).mean()
        session.log_indicator("range", range)

        if self.RANGE < range:
            return

        ohlcv = session.ohlcv(3, 1)         # 3秒足を1本分取得。
        if len(ohlcv) < 1:                 # 3秒間に約定データがない場合リターン
            print("NO OHLCV DATA")
            return

        if session.position <= 0.001:    # ポジションが少ない場合は買い注文を出す。
            order_price = ohlcv['close'][-1] * (1 - self.OFFSET)
            print("BUY ORDER: ", order_price, self.ORDER_SIZE)
            session.limit_order('Buy', order_price, self.ORDER_SIZE - session.position)

        else:                       # ポジションがある場合は売り注文を出す。
            order_price = ohlcv['close'][-1] * (1 + self.OFFSET)
            print("SELL ORDER: ", order_price, session.position)
            session.limit_order('Sell', order_price, session.position)

    def on_update(self, session, updated_order):
        # 売りオーダーが期限切れされた場合には、成り行きで売り注文を出す（ロスカット）
        if updated_order.status == 'Canceled':
            if updated_order.side == 'Sell':
                session.market_order('Sell', updated_order.remaining_size)



## バックテスト

In [4]:
from rbot import Bybit
from rbot import BybitConfig
from rbot import Runner
from rbot import NOW, DAYS

rbot version:  0.2.15
!!! ABSOLUTELY NO WARRANTY !!!
!!!  USE AT YOUR OWN RISK  !!!
See document at https://github.com/yasstake/rusty-bot
 All rights reserved. (c) 2022-2023 rbot developers / yasstake


In [5]:
# Colabの場合、DBの保存先をGoogleDriveにする。
# 以下のように環境変数へ設定するようにする（予定）

#import os, sys

#if 'google.colab' in sys.modules:
#    os.environ['RBOT_DB_ROOT'] = '/content/drive/MyDrive'


In [6]:
bybit = Bybit()

market = bybit.open_market(BybitConfig.BTCUSDT)

# バックテストの期間を設定
BACKTEST_DAYS = 2

# バックテスト用に過去データをダウンロード
market.download(
    ndays=BACKTEST_DAYS,          # 過去2日分のデータをダウンロード
    verbose=True,
    archive_only=True
)

1919013

In [7]:
agent = BasilAgent()        # 実行対象のエージェントを指定
runner = Runner()           # 実行モジュール Runner を作成

In [8]:
session = runner.back_test(
    agent=agent,                    # エージェントを指定
    market=market,                  # マーケットを指定
    start_time=NOW()-DAYS(BACKTEST_DAYS), # 開始時刻を指定
    end_time=0,                     # 終了時刻を指定(0を指定すると最後まで実行)
    # execute_time=60*60*24,        # 実行時間を指定(指定しない、もしくは0を指定すると最後まで実行)
    verbose=True,                   # 実行の進捗を表示
    # log_file="./bot.log"            # ログファイルを指定(指定しないとファイルは作られない）
)


BUY ORDER:  42641.46860000001 0.01
BUY ORDER:  42669.054800000005 0.01
BUY ORDER:  42740.31915 0.01
SELL ORDER:  42757.367999999995 0.01
BUY ORDER:  42738.62 0.01
SELL ORDER:  42759.068849999996 0.01
SELL ORDER:  42748.06335 0.01
SELL ORDER:  42696.83775 0.01
BUY ORDER:  42677.250700000004 0.01
BUY ORDER:  42720.42910000001 0.01
BUY ORDER:  42705.93635 0.01
SELL ORDER:  42726.05235 0.01
BUY ORDER:  42706.2362 0.01
BUY ORDER:  42777.40060000001 0.01
BUY ORDER:  42774.70195 0.01
BUY ORDER:  42756.71095 0.01
BUY ORDER:  42760.009300000005 0.01
SELL ORDER:  42780.979799999994 0.01
SELL ORDER:  42744.46154999999 0.01
BUY ORDER:  42723.4276 0.01
BUY ORDER:  42739.719450000004 0.01
BUY ORDER:  42735.6215 0.01
SELL ORDER:  42751.064849999995 0.01
SELL ORDER:  42729.754199999996 0.01
BUY ORDER:  42708.635 0.01
SELL ORDER:  42730.454549999995 0.01
BUY ORDER:  42726.52605 0.01
BUY ORDER:  42769.6045 0.01
SELL ORDER:  42787.7832 0.01
SELL ORDER:  42731.955299999994 0.01
BUY ORDER:  42711.933350000

In [9]:
log = session.log
orders = log.orders
orders.tail(5)

log_id,symbol,update_time,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change,open_position,close_position,position,profit,fee,total_profit,sum_profit
i64,str,datetime[μs],datetime[μs],str,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,str,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1056,"""BTCUSDT""",2023-12-27 23:52:01.075,2023-12-27 23:51:03,"""Filled""","""20231228T1124-…","""20231228T1124-…","""Buy""","""Limit""",43417.88,0.01,0.0,"""20231228T1124-…",43417.88,0.008,347.34304,0.0,"""""",true,"""""",0.0,0.0,-347.34304,0.008,0.0,0.008,-347.34304,0.0,0.008,0.0,0.01,0.0,0.034734,-0.034734,-10.650507
1057,"""BTCUSDT""",2023-12-27 23:52:03.772100,2023-12-27 23:52:03,"""New""","""20231228T1124-…","""20231228T1124-…","""Sell""","""Limit""",43434.706,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""""",true,"""""",0.0,0.0,0.0,0.0,0.0,-0.01,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,-10.650507
1058,"""BTCUSDT""",2023-12-27 23:58:27.193400,2023-12-27 23:52:03,"""PartiallyFille…","""20231228T1124-…","""20231228T1124-…","""Sell""","""Limit""",43434.706,0.01,0.009,"""20231228T1124-…",43434.706,0.001,43.434706,0.0,"""""",true,"""""",0.0,0.0,43.434706,-0.001,43.434706,-0.001,0.0,-0.001,0.0,0.001,0.009,0.016826,0.004343,0.012483,-10.638024
1059,"""BTCUSDT""",2023-12-27 23:58:27.193400,2023-12-27 23:52:03,"""Filled""","""20231228T1124-…","""20231228T1124-…","""Sell""","""Limit""",43434.706,0.01,0.0,"""20231228T1124-…",43434.706,0.009,390.912354,0.0,"""""",true,"""""",0.0,0.0,390.912354,-0.009,390.912354,-0.009,0.0,-0.009,0.0,0.009,0.0,0.151434,0.039091,0.112343,-10.525681
1060,"""BTCUSDT""",2023-12-27 23:58:30.157500,2023-12-27 23:58:30,"""New""","""20231228T1124-…","""20231228T1124-…","""Buy""","""Limit""",43419.379,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""""",true,"""""",0.0,0.0,0.0,0.0,-434.19379,0.0,434.19379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.525681


In [10]:
# オーダー状況分析
# あまりにCancelが多い場合は、Expireの時間やオーダーのOFFSETを調整する必要がある。
orders.group_by(['order_side', 'status']).count()

order_side,status,count
str,str,u32
"""Buy""","""New""",184
"""Buy""","""PartiallyFille…",174
"""Buy""","""Filled""",129
"""Sell""","""Filled""",129
"""Sell""","""New""",183
"""Sell""","""Canceled""",54
"""Buy""","""Canceled""",54
"""Sell""","""PartiallyFille…",153


In [11]:
# 最終想定損益
orders['sum_profit'][-1]

-10.525681462000003

In [12]:
# 手数料なし想定損益
orders['profit'].sum()

0.5278999999999996

In [13]:

# ローソク足データの作成
ohlcv = market.ohlcv(
    runner.start_timestamp,     # バックテストの開始時刻
    runner.last_timestamp,    # バックテストの終了時刻
    30                          # 30秒足
)
ohlcv.tail(5)

timestamp,open,high,low,close,volume,count
datetime[μs],f64,f64,f64,f64,f64,u32
2023-12-27 23:57:30.663700,43415.9,43429.2,43414.7,43423.3,1.745,77
2023-12-27 23:58:00.663700,43423.3,43441.7,43412.2,43441.7,32.162,253
2023-12-27 23:58:30.663700,43441.6,43459.9,43441.6,43456.1,37.498,435
2023-12-27 23:59:00.663700,43456.1,43459.9,43456.0,43456.0,10.176,102
2023-12-27 23:59:30.663700,43456.1,43459.8,43450.7,43454.0,11.396,113


In [14]:
# session.log_indicator('range', value)で保存したデータは以下で取得できます。戻り値はpolars.DataFrameです。
log['range'].tail(5)

timestamp,range
datetime[μs],f64
2023-12-27 23:42:21,280.06
2023-12-27 23:45:36,280.06
2023-12-27 23:51:03,276.42
2023-12-27 23:52:03,265.72
2023-12-27 23:58:30,257.92


In [15]:
# バックテスト結果の可視化

# 数千件のデータを可視化するときは、plotlyを使うと便利。
# https://plotly.com/python/
# ただし約定データが万を超えると、ブラウザが重くなるので注意。一部データ切り取りなどが必要。

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("range", "psudo-position", "psudo-profit", "candlestick"))

# row 1 (indicator)
spread = log['range']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['range'], name="range"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='red', size=10), name="buy canceled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='blue', size=10), name="sell canceled"), row=4, col=1)

fig.update_layout(height=800, title_text="Backtest Result")


# フォーワードテスト

ここからは、WebSocketでリアルタイムデータを使います。そのためColabでは動きません。ローカルにjupyter環境を準備して実行してください。

In [ ]:
runner = Runner()

session = runner.dry_run(
    agent=agent,
    market=market,
    log_memory=True,
    execute_time=60*5,     # 60x5=5分間
    verbose=True,
    # log_file="./bot.log"            # ログファイルを指定(指定しないとファイルは作られない）
)

In [ ]:
log = session.log

In [ ]:
orders = log.orders
orders.tail(5)

In [ ]:
ohlcv = market.ohlcv(runner.start_timestamp, runner.last_timestamp, 10)
ohlcv

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("range", "psudo-position", "psudo-profit", "candlestick"))

# row 1 (indicator)
spread = log['range']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['range'], name="range"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='red', size=10), name="buy canceled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='blue', size=10), name="sell canceled"), row=4, col=1)

fig.update_layout(height=800, title_text="Forwadtest Result")


In [ ]:
market.ohlcv(0, 0, 600)